In [1]:
import sys
sys.path.append('/dccstor/hoo-misha-1/wilds/WOODS')
#sys.path.append('/dccs|tor/hoo-misha-1/wilds/WOODS/scripts/')
sys.path.append('/dccstor/hoo-misha-1/wilds/wilds')
import numpy as np
import torch
import os

import argparse

from scripts.utils import *

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='breeds', help='Dataset name')
parser.add_argument('--model', type=str, default="deepCORAL", help="Model name to featurize dataset, not all models for a dataset, Default: ERM")
parser.add_argument('--model_name', type=str, help="Model name to featurize dataset")
parser.add_argument('--no_rng', action='store_true', help='Toggles fixed RNG generator for consistency, Default: False') 
parser.add_argument('--cutoff', type=int, default=25, help='Cutoff for number of shots to test, Default: 25')
sampling_algorithms_dict = {'iterative':'iterative_kmeans_sample_ind', 'weighted':'weighted_iterative_kmeans_sample_ind', 'dense':'dense'}
parser.add_argument('--sampler', type=str, choices=sampling_algorithms_dict.keys(), default='iterative', help='Sampling algorithms to select training data, Default: balanced')
parser.add_argument('--precluster', type=bool, default = False)
parser.add_argument('--typicality', type=bool, default = False)
parser.add_argument('--reset', type=bool, default = False)

parser.add_argument('--features', type=str, help='Optional path to featurize data of the dataset/model specified')
parser.add_argument('--classifier', type=str, help='Optional path to the original classifier of the dataset/model specified')
parser.add_argument('--results', type=str, default='/dccstor/hoo-misha-1/wilds/WOODS/results', help='Optional path to store accuracy results')

_StoreAction(option_strings=['--results'], dest='results', nargs=None, const=None, default='/dccstor/hoo-misha-1/wilds/WOODS/results', type=<class 'str'>, choices=None, help='Optional path to store accuracy results', metavar=None)

In [21]:
config = parser.parse_args('--no_rng'.split())

In [22]:
if config.model_name is None:
    if config.dataset == 'iwildcam':
        config.model_name = 'resnet50'
    elif config.dataset == 'camelyon17':
        config.model_name = 'densenet121'
    elif config.dataset == 'breeds':
        config.model_name = 'resnet50'

set_model_name(config.model_name)

if config.features is None:
    config.features = f'/dccstor/hoo-misha-1/wilds/wilds/features/{config.dataset}/{config.model}'

set_path_base(config.features)

if config.classifier is None:
    config.classifier = f'/dccstor/hoo-misha-1/wilds/WOODS/classifiers/{config.dataset}/{config.dataset}_{config.model}_classifier'

if config.no_rng:
    rng = np.random.default_rng(2022)
    random_state = 0
else:
    rng = np.random.default_rng()
    random_state = None
set_random_state(rng, random_state)

set_original_classifier(config.classifier)

set_config(config)

In [23]:
f,l,m = cam_flm(1, [0])

In [24]:
unique_classes = np.unique(l)
num_classes = len(unique_classes)

In [25]:
feature_centers, centers = iterative_kmeans_sample_ind_helper(f,num_classes,num_shots=25)

In [26]:
feature_centers_1, centers_1 = iterative_kmeans_sample_ind_helper(f,num_classes,num_shots=25)

In [27]:
(centers == centers_1).all()

False

In [28]:
x = np.random.random((100,2048))

In [29]:
kmeans = get_kmeans(x,25)
kmeans_1 = get_kmeans(x,25)

In [30]:
centers = kmeans.cluster_centers_
centers_1 = kmeans_1.cluster_centers_

In [31]:
(centers == centers).all()

True

In [32]:
(centers_1 == centers_1).all()

True

In [33]:
(centers == centers_1).all()

True

In [48]:
sampled_ind_1 = iterative_kmeans_sample_ind(f, l, num_shots=5, n=100)

[14953, 3792, 48, 3125, 7299, 16191, 7916, 5028, 11745, 1522, 1995, 6492, 2745, 6841, 15386, 6531, 6328, 12017, 6060, 330, 16343, 5122, 8040, 5136, 9881, 7417, 5137, 7801, 9843, 15223, 2127, 13981, 171, 10015, 4457, 8043, 4730, 2905, 1354, 16366, 7296, 7155, 7315, 16344, 10375, 6335, 5357, 8796, 8520, 14185, 2701, 8916, 8722, 3457, 6632, 2717, 784, 13991, 523, 4193, 13171, 9345, 1631, 12372, 9116, 12312, 9344, 3438, 5504, 805, 13864, 5356, 1058, 7334, 1502, 13534, 11887, 15311]


In [49]:
sampled_ind_1

array([14953,  3792,    48,  3125,  7299, 16191,  7916,  5028, 11745,
        1522,  1995,  6492,  2745,  6841, 15386,  6531,  6328, 12017,
        6060,   330, 16343,  5122,  8040,  5136,  9881,  7417,  5137,
        7801,  9843, 15223,  2127, 13981,   171, 10015,  4457,  8043,
        4730,  2905,  1354, 16366,  7296,  7155,  7315, 16344, 10375,
        6335,  5357,  8796,  8520, 14185,  2701,  8916,  8722,  3457,
        6632,  2717,   784, 13991,   523,  4193, 13171,  9345,  1631,
       12372,  9116, 12312,  9344,  3438,  5504,   805, 13864,  5356,
        1058,  7334,  1502, 13534, 11887, 15311])

In [35]:
feature_centers, centers = iterative_kmeans_sample_ind_helper(f,num_classes,num_shots=25)
mask = np.ones((feature_centers.shape[0],1))
feature_mask = np.zeros((1,f.shape[0]))

centers_features_pairwise_dist = e_dist(centers, f)
sampled_ind = np.argmin(centers_features_pairwise_dist,1)
feature_mask[0,sampled_ind] = np.inf

if config.sampler in ['weighted', 'dense']:
    center_labels = list(l[sampled_ind])
    center_label_uniques, center_label_counts = np.unique(center_labels, return_counts = True)
    center_label_count_dict = dict(zip(center_label_uniques, center_label_counts))
else:
    center_labels = None
    center_label_count_dict = None

if config.sampler == 'dense':
    feature_center_density = e_dist(feature_centers, feature_centers)
    gaussian = scipy.stats.norm(0, 1)
    feature_center_density = gaussian.pdf(feature_center_density)
    feature_center_density = np.sum(feature_center_density, 1, keepdims=True)
else:
    feature_center_density = None

In [43]:
for i in range(num_classes):
            centers, mask = progressive_iterative_kmeans_sample_ind(feature_centers, centers, mask, config.sampler, center_labels = center_labels, center_label_count_dict = center_label_count_dict, feature_center_density = feature_center_density)
            newest_center = centers[[-1]]

            newest_center_features_pairwise_dist = e_dist(newest_center, f) + feature_mask
            newest_ind = np.argmin(newest_center_features_pairwise_dist)

            feature_mask[0,newest_ind] = np.inf
            sampled_ind = np.append(sampled_ind,newest_ind)

            new_center_label = l[newest_ind]
            if center_labels is not None:
                center_labels.append(new_center_label)
                if new_center_label in center_label_count_dict:
                    center_label_count_dict[new_center_label] += 1
                else:
                    center_label_count_dict[new_center_label] = 1

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 2048)

In [57]:
sampled_ind_1

array([14953,  3792,    48,  3125,  7299, 16191,  7916,  5028, 11745,
        1522,  1995,  6492,  2745,  6841, 15386,  6531,  6328, 12017,
        6060,   330, 16343,  5122,  8040,  5136,  9881,  7417])

In [67]:
set(sampled_ind_1) & set(sampled_ind)

{1522, 1995, 2745, 3125, 5028, 6531, 6841, 7299, 7916, 11745, 14953, 15386}